# <center> 📝 Пишем скрипт для которого запустим подбор гипер-параметров в ClearML. 🤹‍♂️

# Что будет в этом ноуте?!

<div class="alert alert-info">
    
* Создадим простой скрипт обучения нейросети для классификации изображений
* Возьмём датасет `CIFAR10`
* Воспользуемся `Pytorch Lightning` для обучения и залогируем всё в `ClearML`
* Далее перенесём весь код в `.py` файл.

# Что важно учесть?!

<div class="alert alert-info">

Чтобы впоследствии запустить подбор гиперпараметров необходимо:
    
* Залогировать эксперимент в `ClearML`
* Лучше запускать скрипт в изолированном виртуальном окружении. 
* Убрать из скрипта весь пользовательский ввод
* Добавить в скрипте логирование метрики, которую будем улучшать

### Настраиваем всё для обучения

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

from lightning import Trainer, LightningModule, LightningDataModule
from clearml import Task

In [2]:
# Initialize ClearML task and configuration
task = Task.init(
    project_name="Image Example", task_name="CIFAR10 Lightning with DataModule"
)

ClearML Task: created new task id=21ca76702bec463a9430055bbb44b980
2025-03-26 20:49:47,719 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/8c64c7258641469bbe1d0988419a80b7/experiments/21ca76702bec463a9430055bbb44b980/output/log


In [3]:
config = {
    "num_epochs": 2,
    "batch_size": 4,
    "dropout": 0.25,
    "learning_rate": 0.001,
    "num_workers": 2,
    "data_dir": "./data",
}
config = task.connect(config) # <------ обязательный элемент для дальнешего перебора. 
print(config)

{'num_epochs': 2, 'batch_size': 4, 'dropout': 0.25, 'learning_rate': 0.001, 'num_workers': 2, 'data_dir': './data'}


In [4]:
# Пишем LightningDataModule для CIFAR10
class CIFAR10DataModule(LightningDataModule):
    def __init__(self, data_dir, batch_size, num_workers=2):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = transforms.Compose([transforms.ToTensor()])

    def prepare_data(self):
        # Download data if not already available
        datasets.CIFAR10(root=self.data_dir, train=True, download=True)
        datasets.CIFAR10(root=self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Setup datasets for train, validation, and test
        if stage == "fit" or stage is None:
            self.trainset = datasets.CIFAR10(
                root=self.data_dir, train=True, transform=self.transform
            )
            self.valset = datasets.CIFAR10(
                root=self.data_dir, train=False, transform=self.transform
            )
        if stage == "test" or stage is None:
            self.testset = datasets.CIFAR10(
                root=self.data_dir, train=False, transform=self.transform
            )

    def train_dataloader(self):
        return DataLoader(
            self.trainset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.valset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.testset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

In [8]:
# Пишем LightningModule дял классификации CIFAR10
class LitCIFAR10Classifier(LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters(config)
        self.test_accs = []
        # Define the network architecture
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.dropout = nn.Dropout(p=self.hparams.get("dropout", 0.25))
        self.fc3 = nn.Linear(84, 10)
        self.criterion = nn.CrossEntropyLoss()
        self.classes = (
            "plane",
            "car",
            "bird",
            "cat",
            "deer",
            "dog",
            "frog",
            "horse",
            "ship",
            "truck",
        )

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(self.dropout(x))
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True)
        
        return {"val_loss": loss, "val_acc": acc}

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.test_accs.append(acc)
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        return {"test_loss": loss, "test_acc": acc}

    def on_test_epoch_end(self):
        # Compute average test accuracy
        avg_acc = sum(self.test_accs) / len(self.test_accs)
        self.log("test_accuracy", avg_acc)

        # Log accuracy in ClearML
        task.get_logger().report_single_value(name="ACC", value=avg_acc.item())
        task.get_logger().report_scalar(
            "Test Accuracy", "Final", avg_acc.item(), iteration=0
        )
        print(f"Final Test Accuracy: {avg_acc.item():.4f}")

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(), lr=self.hparams.get("learning_rate", 0.001), momentum=0.9
        )
        return optimizer

In [9]:
# Инициализируем DataModule and LightningModule
data_module = CIFAR10DataModule(
    data_dir=config.get("data_dir", "./data"),
    batch_size=config.get("batch_size", 4),
    num_workers=config.get("num_workers", 2),
)
model = LitCIFAR10Classifier(config)

In [10]:
# Создаём Trainer
trainer = Trainer(max_epochs=config.get("num_epochs", 3))
trainer.fit(model, datamodule=data_module)
trainer.test(model, datamodule=data_module)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | conv1     | Conv2d           | 168    | train
1 | conv2     | Conv2d           | 880    | train
2 | pool      | MaxPool2d        | 0      | train
3 | fc1       | Linear           | 69.2 K | train
4 | fc2       | Linear           | 10.2 K | train
5 | dropout   | Dropout          | 0      | train
6 | fc3       | Linear           | 850    | train
7 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
81.3 K    Trainable params
0         Non-trainable params
81.3 K    Total params
0.325     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

Final Test Accuracy: 0.5032


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5031999945640564     │
│       test_accuracy       │    0.5031999945640564     │
│         test_loss         │     1.365828275680542     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.365828275680542,
  'test_acc': 0.5031999945640564,
  'test_accuracy': 0.5031999945640564}]

In [11]:
# Сохраняем веса модели
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

2025-03-26 20:54:30,480 - clearml.frameworks - INFO - Found existing registered model id=bdf7799c7e6d4372a3ddc39c151ac01b [/app/storage_local/Course/ml_instruments/notebooks/cifar_net.pth] reusing it.


<div class="alert alert-info">
    
**Важно!** Получить `ID` эксперимента для запуска подбора.
* Можно посмотреть его при инициализации `Task`
* Можно скопировать из `WebUI`
* Удобнее всего вывести отдельно в конце скрипта через `task.id`


In [12]:
print("Task ID number is: {}".format(task.id))

Task ID number is: 21ca76702bec463a9430055bbb44b980


In [13]:
# Не забываем завершить эксперимент
task.close()

<div class="alert alert-info">

Теперь создадим `.py` файл с кодом обучения.

In [1]:
%%writefile train_script.py
#!/usr/bin/env python

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

from lightning import Trainer, LightningModule, LightningDataModule
from clearml import Task

# Initialize ClearML task and configuration
task = Task.init(
    project_name="Image Example", task_name="CIFAR10 Lightning with DataModule"
)


config = {
    "num_epochs": 2,
    "batch_size": 4,
    "dropout": 0.25,
    "learning_rate": 0.001,
    "num_workers": 2,
    "data_dir": "./data",
}
config = task.connect(config)
#print(config)


# Define the LightningDataModule for CIFAR10
class CIFAR10DataModule(LightningDataModule):
    def __init__(self, data_dir, batch_size, num_workers=2):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = transforms.Compose([transforms.ToTensor()])

    def prepare_data(self):
        # Download data if not already available
        datasets.CIFAR10(root=self.data_dir, train=True, download=True)
        datasets.CIFAR10(root=self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Setup datasets for train, validation, and test
        if stage == "fit" or stage is None:
            self.trainset = datasets.CIFAR10(
                root=self.data_dir, train=True, transform=self.transform
            )
            self.valset = datasets.CIFAR10(
                root=self.data_dir, train=False, transform=self.transform
            )
        if stage == "test" or stage is None:
            self.testset = datasets.CIFAR10(
                root=self.data_dir, train=False, transform=self.transform
            )

    def train_dataloader(self):
        return DataLoader(
            self.trainset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.valset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.testset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

# Define the LightningModule for the CIFAR10 classifier
class LitCIFAR10Classifier(LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters(config)
        self.test_accs = []
        # Define the network architecture
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.dropout = nn.Dropout(p=self.hparams.get("dropout", 0.25))
        self.fc3 = nn.Linear(84, 10)
        self.criterion = nn.CrossEntropyLoss()
        self.classes = (
            "plane",
            "car",
            "bird",
            "cat",
            "deer",
            "dog",
            "frog",
            "horse",
            "ship",
            "truck",
        )

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(self.dropout(x))
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True)
        task.get_logger().report_scalar(
            title="val_acc",
            series="val_acc",
            value=acc.item(),
            iteration=self.current_epoch,
        )
        return {"val_loss": loss, "val_acc": acc}

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.test_accs.append(acc)
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        return {"test_loss": loss, "test_acc": acc}

    def on_test_epoch_end(self):
        # Compute average test accuracy
        avg_acc = sum(self.test_accs) / len(self.test_accs)
        self.log("test_accuracy", avg_acc)

        # Log accuracy in ClearML
        task.get_logger().report_single_value(name="ACC", value=avg_acc.item())
        task.get_logger().report_scalar(
            "Test Accuracy", "Final", avg_acc.item(), iteration=0
        )
        print(f"Final Test Accuracy: {avg_acc.item():.4f}")

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(), lr=self.hparams.get("learning_rate", 0.001), momentum=0.9
        )
        return optimizer


# Instantiate the DataModule and LightningModule
data_module = CIFAR10DataModule(
    data_dir=config.get("data_dir", "./data"),
    batch_size=config.get("batch_size", 4),
    num_workers=config.get("num_workers", 2),
)
model = LitCIFAR10Classifier(config)


# Create the Trainer using the new Lightning import and run training
trainer = Trainer(max_epochs=config.get("num_epochs", 3))
trainer.fit(model, datamodule=data_module)
trainer.test(model, datamodule=data_module)


# Save the trained model weights
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)


print("Task ID number is: {}".format(task.id))


task.close()

Writing train_script.py


In [2]:
# Тут же можем проверить его работу
!python train_script.py

ClearML Task: created new task id=519b5fe1851e4a93889e59efdb145e44
ClearML results page: https://app.clear.ml/projects/8c64c7258641469bbe1d0988419a80b7/experiments/519b5fe1851e4a93889e59efdb145e44/output/log
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
--------------

<div class="alert alert-success">
    
# <center> 🏃‍♂️ Теперь переходим в следующий ноутбук, где разберёмся как запустить подбор гиперпараметров!